In [9]:
x = ezdxf.readfile("../dxffiles/F15-TSR-TSM-Master-04.dxf")
for i in x.entities:
    if i.dxftype() == "LINE":
        break

In [36]:
ezdxf.document.const.versions_supported_by_new
dir(ezdxf.document.const)
#ezdxf.document.const.acad_release
ezdxf.document.const.versions_supported_by_new+list(ezdxf.document.const.acad_release_to_dxf_version.keys())



['AC1009',
 'AC1015',
 'AC1018',
 'AC1021',
 'AC1024',
 'AC1027',
 'AC1032',
 'R9',
 'R10',
 'R12',
 'R13',
 'R14',
 'R2000',
 'R2004',
 'R2007',
 'R2010',
 'R2013',
 'R2018']

In [28]:
ezdxf.new("R13")

DXFVersionError: Unsupported DXF version "AC1012".

In [21]:
help(ezdxf.new)

Help on function new in module ezdxf.filemanagement:

new(dxfversion: str = 'AC1027', setup: Union[str, bool, Sequence[str]] = False, units: int = 6) -> 'Drawing'
    Create a new :class:`~ezdxf.drawing.Drawing` from scratch, `dxfversion`
    can be either "AC1009" the official DXF version name or "R12" the
    AutoCAD release name.
    
    :func:`new` can create drawings for following DXF versions:
    
    ======= ========================
    Version AutoCAD Release
    ======= ========================
    AC1009  AutoCAD R12
    AC1015  AutoCAD R2000
    AC1018  AutoCAD R2004
    AC1021  AutoCAD R2007
    AC1024  AutoCAD R2010
    AC1027  AutoCAD R2013
    AC1032  AutoCAD R2018
    ======= ========================
    
    The `units` argument defines th document and modelspace units. The header
    variable $MEASUREMENT will be set according to the given `units`, 0 for
    inch, feet, miles, ... and 1 for metric units. For more information go to
    module :mod:`ezdxf.units`
    


In [17]:

dir(i.dxf)

['__class__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_entity',
 '_export_dxf_attribute_optional',
 'all_existing_dxf_attribs',
 'color',
 'copy',
 'discard',
 'dxf_default_value',
 'dxfattribs',
 'dxftype',
 'end',
 'export_dxf_attribs',
 'get',
 'get_default',
 'handle',
 'hasattr',
 'is_supported',
 'layer',
 'linetype',
 'lineweight',
 'owner',
 'reset_handles',
 'rewire',
 'set',
 'start',
 'unprotected_set',
 'update']

In [15]:
ezdxf.math.Vec3.from_deg_angle(90)

Vec3(6.123233995736766e-17, 1.0, 0.0)

In [9]:
import os, ezdxf

# possibly merge all the 170color files into one (instead of by directory)
dfiles = "../dxffiles/aamaout/170color"
dfs = [ ]
for f in os.listdir(dfiles):
    if f[0] != "#" and f[0] != "p" and os.path.splitext(f)[1] == ".dxf":
        df = os.path.join(dfiles, f)
        dfs.append(df)

In [12]:
from dxfgrouping import linearizeelement
import json

js = [ ]
for df in dfs:
    d = ezdxf.readfile(df)
    for b in d.entities:
        assert b.dxftype() == "INSERT"
        j = { "blockname":b.dxf.name, "blockrotation":b.dxf.rotation, "blockposition":list(b.dxf.insert)[:2] }
        bb = d.blocks[b.dxf.name]
        outercut = bb.entity_space.entities[0]
        assert outercut.dxftype() == "POLYLINE"
        assert outercut.dxf.layer == "1"
        outercontour = [ list(v.dxf.location)[:2]  for v in outercut.vertices ]
        assert outercontour[0] == outercontour[-1]
        j["outercontour"] = outercontour
        innerlines = [ ]
        for e in bb.entity_space.entities[1:]:
            innerlines.append([ list(p)[:2]  for p in linearizeelement(e) ])
        j["innerlines"] = innerlines
        js.append(j)
        

In [14]:
json.dump(js, open(os.path.join(dfiles, "shapes.json"), "w"))

In [35]:
jps = json.load(open(os.path.join(dfiles, "positions.json"), "r"))
outputfilename = os.path.join(dfiles, "positions.dxf")
doc = ezdxf.new('R12')
msp = doc.modelspace()
aamacutlayer = doc.layers.new("1", {"color":1})
aamadrawlayer = doc.layers.new("8", {"color":4})
aamaintcutlayer = doc.layers.new("11", {"color":3})
for df in dfs:
    d = ezdxf.readfile(df)
    for b in d.entities:
        assert b.dxftype() == "INSERT"
        bb = d.blocks[b.dxf.name]
        block = doc.blocks.new(name=b.dxf.name)
        for e in bb.entity_space.entities:
            block.add_foreign_entity(e)
        bp = jps[b.dxf.name]
        dxfattribs = {'rotation':bp[2], 'linetype':'BYLAYER' }
        msp.add_blockref(b.dxf.name, ezdxf.math.vector.Vector(bp[0], bp[1], 0), dxfattribs=dxfattribs)
        
    doc.set_modelspace_vport(height=2300, center=(1800, 900))
    doc.saveas(outputfilename)
